## Callin Switzer
## Try to follow a trajectory, based on nnet

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import os
import pandas as pd
import time
from datetime import datetime
import sys
import csv
import seaborn as sns
sns.set(style="ticks")
import scipy.io
import subprocess
import csv
import pickle

import tensorflow as tf
print("TensorFlow successfully installed.")
if tf.test.is_built_with_cuda():
    print("The installed version of TensorFlow includes GPU support.")

print(sys.version, "\n")

now = datetime.now()
print("last run on " + str(now))

#%qtconsole

# define directories
baseDir = os.getcwd()
dataDir = r'D:\MothSimulations\11c-AggressiveManeuver\Qstore\hws_am_con'
figDir = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\Figs'
dataOutput = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\DataOutput'
savedModels = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\savedModels'
randomRawData = r'D:/Dropbox/AcademiaDropbox/mothMachineLearning_dataAndFigs/PythonGeneratedData'

if not os.path.exists(dataOutput):
    os.mkdir(dataOutput)
if not os.path.exists(savedModels):
    os.mkdir(savedModels)
    
# open file explorer
# aa = r'explorer /select,"'
# subprocess.Popen(str(aa + figDir + '\"'))
# subprocess.Popen(str(aa + dataDir + '\"'))

C:\Users\calli\Anaconda3\envs\deeplearning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow successfully installed.
The installed version of TensorFlow includes GPU support.
3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)] 

last run on 2019-01-01 21:50:03.252415


In [2]:
# import custom Utils functions
from DeepLearningUtils import *

TensorFlow successfully installed.
The installed version of TensorFlow includes GPU support.
3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)] 

last run on 2019-01-01 21:50:03.259366


In [3]:
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.models import Model
from keras import regularizers

# Keras callcacks
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras.backend as K

Using TensorFlow backend.


In [4]:
# Load model
K.clear_session()
from keras.models import load_model
model = load_model(os.path.join(savedModels, 'my_model_400Units.h5'))

In [5]:
# Generate path to follow
# 2 Hz sine wave, points 0.02 sec apart
tsteps = np.linspace(0, 0.5, num = 26)
sig = 10 * np.sin(2*np.pi* 2*tsteps)

In [13]:
# Predict steps with nnet (without feedback)
x_0 = np.repeat(0, len(tsteps))
x_f = x_0.copy()
y_0 = x_0.copy()
y_error = np.hstack([np.nan, np.diff(sig)])
theta_0 = np.repeat(0.785398, len(tsteps))
phi_0 = np.repeat(3.926991, len(tsteps))
x_dot_0 = np.repeat(0.0001, len(tsteps))
y_dot_0 = np.repeat(0.0001, len(tsteps))
phi_dot_0 = np.repeat(0.0001, len(tsteps))
theta_dot_0 = np.repeat(0.0001, len(tsteps))
Fx = np.repeat(0, len(tsteps))
Fy = np.repeat(0, len(tsteps))
tau = np.repeat(0, len(tsteps))


predDF = pd.DataFrame(np.transpose(np.array([tsteps, 
                                             sig, 
                                             x_0,
                                             x_f,
                                             y_0,
                                             y_error,
                                             theta_0,
                                             theta_0,
                                             phi_0, 
                                             phi_0, 
                                             x_dot_0, 
                                             y_dot_0, 
                                             phi_dot_0, 
                                             theta_dot_0, 
                                             Fx, 
                                             Fy, 
                                             tau
                                            ])), 
                     columns=["tsteps", "signal", "x_0", "x_99", "y_0", "y_99", 
                             "theta_0", "theta_99", "phi_0", "phi_99", 
                             "x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0", 
                             "Fx", "Fy", "tau_pred"])



predDF["x_99_actual"] = 0
predDF["xd_99_actual"] = 0
predDF["y_99_actual"] = 0
predDF["yd_99_actual"] = 0

predDF["theta_99_actual"] = 0
predDF["thetad_99_actual"] = 0

predDF["phi_99_actual"] = 0
predDF["phid_99_actual"] = 0

predDF["F_pred"] = 0
predDF["alpha_pred"] = 0


pd.set_option('display.max_columns', 500)
predDF.head()

# X = testDF.loc[:, ["x_0", "y_0", "phi_0", "theta_0", 
#                     "x_99", "y_99", "phi_99", "theta_99", 
#                    "x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"]]

,tsteps,signal,x_0,x_99,y_0,y_99,theta_0,theta_99,phi_0,phi_99,x_dot_0,y_dot_0,phi_dot_0,theta_dot_0,Fx,Fy,tau_pred,x_99_actual,xd_99_actual,y_99_actual,yd_99_actual,theta_99_actual,thetad_99_actual,phi_99_actual,phid_99_actual,F_pred,alpha_pred
0,0.00,0.000000,0.0,0.0,0.0,NaN,0.785398,0.785398,3.926991,3.926991,0.0001,0.0001,0.0001,0.0001,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0
1,0.02,2.486899,0.0,0.0,0.0,2.486899,0.785398,0.785398,3.926991,3.926991,0.0001,0.0001,0.0001,0.0001,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0
2,0.04,4.817537,0.0,0.0,0.0,2.330638,0.785398,0.785398,3.926991,3.926991,0.0001,0.0001,0.0001,0.0001,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0
3,0.06,6.845471,0.0,0.0,0.0,2.027934,0.785398,0.785398,3.926991,3.926991,0.0001,0.0001,0.0001,0.0001,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0
4,0.08,8.443279,0.0,0.0,0.0,1.597808,0.785398,0.785398,3.926991,3.926991,0.0001,0.0001,0.0001,0.0001,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0


In [14]:
# scale data for nnet predictions
X = predDF.loc[1:, ["x_0", "y_0", "phi_0", "theta_0", 
                    "x_99", "y_99", "phi_99", "theta_99", 
                   "x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"]]

# scale data
scalerfileX = 'scalerX.pkl'
scalerfileY = 'scalerY.pkl'

scalerX = pickle.load(open(os.path.join(dataOutput, scalerfileX), 'rb'))
X_scaled = scalerX.transform(X)

scalerY = pickle.load(open(os.path.join(dataOutput, scalerfileY), 'rb'))

In [15]:
# predict steps one at a time
rowNum = 1

nnpreds = model.predict(X_scaled[rowNum, :].reshape(-1,12))
nnpreds_unscaled = scalerY.inverse_transform(nnpreds)

# Y = testDF.loc[:, ["Fx", "Fy", "tau", "x_dot_99", "y_dot_99", 
#                    "phi_dot_99", "theta_dot_99"] ]


# these are input values
##         x,    xd,    y,   yd,    theta,  thetad,       phi,     phid,         F,          alpha,       tau0
print(np.array(X.loc[rowNum, ["x_0", "x_dot_0", "y_0", "y_dot_0", "theta_0", "theta_dot_0", "phi_0", "phi_dot_0"]]))

# this is for the next step -- they are predictions
print(pd.DataFrame(np.array(nnpreds_unscaled[0, [3,4,5,6]]).reshape(1, -1), columns =["x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"]))


[0.000000e+00 1.000000e-04 0.000000e+00 1.000000e-04 7.853980e-01
 1.000000e-04 3.926991e+00 1.000000e-04]
     x_dot_0     y_dot_0  phi_dot_0  theta_dot_0
0 -89.186897  261.604553  -0.066532    -0.632923


In [18]:
PredControl = pd.DataFrame(np.array(nnpreds_unscaled[0, [0,1,2]]).reshape(1, -1), columns = ["Fx", "Fy", "tau_pred"])
# convert to F, alpha, tau
PredControl["F_pred"] = np.sqrt(PredControl["Fx"]**2 + PredControl["Fy"]**2)

# calculate alpha
def quadrant(Fx, Fy):
    if (Fx >= 0) & (Fy >= 0):
        q = 1
    elif (Fx < 0) & (Fy >= 0):
        q = 2
    elif (Fx < 0) & (Fy < 0):
        q = 3
    elif (Fx >= 0) & (Fy < 0):
        q = 4
    else:
        q = 999999
    return(q)

PredControl["quadrant"] = [quadrant(fx, fy) for fx,fy in zip(PredControl["Fx"], PredControl["Fy"])]

def angleCalc(Fx, Fy, q):
    fx = np.abs(Fx)
    fy = np.abs(Fy)
    
    if q == 1:
        alpha = np.arctan(fy/fx)
    elif q == 2:
        alpha = np.pi - np.arctan(fy/fx)
    elif q == 3: 
        alpha = np.pi + np.arctan(fy/fx)
    elif q == 4:
        alpha = (2*np.pi) - np.arctan(fy/fx)
    return(alpha)

PredControl["alpha_pred"] = [angleCalc(fx, fy, q) for fx, fy, q in zip(PredControl["Fx"], PredControl["Fy"], PredControl["quadrant"])]

predDF.loc[[rowNum], ["F_pred", "alpha_pred", "tau_pred"]] = np.array(PredControl.loc[:, ["F_pred", "alpha_pred", "tau_pred"]])
predDF.head()

,tsteps,signal,x_0,x_99,y_0,y_99,theta_0,theta_99,phi_0,phi_99,x_dot_0,y_dot_0,phi_dot_0,theta_dot_0,Fx,Fy,tau_pred,x_99_actual,xd_99_actual,y_99_actual,yd_99_actual,theta_99_actual,thetad_99_actual,phi_99_actual,phid_99_actual,F_pred,alpha_pred
0,0.00,0.000000,0.0,0.0,0.0,NaN,0.785398,0.785398,3.926991,3.926991,0.0001,0.0001,0.0001,0.0001,0.0,0.0,0.000000,0,0,0,0,0,0,0,0,0.00000,0.000000
1,0.02,2.486899,0.0,0.0,0.0,2.486899,0.785398,0.785398,3.926991,3.926991,0.0001,0.0001,0.0001,0.0001,0.0,0.0,-3468.092041,0,0,0,0,0,0,0,0,14022.81543,1.174873
2,0.04,4.817537,0.0,0.0,0.0,2.330638,0.785398,0.785398,3.926991,3.926991,0.0001,0.0001,0.0001,0.0001,0.0,0.0,0.000000,0,0,0,0,0,0,0,0,0.00000,0.000000
3,0.06,6.845471,0.0,0.0,0.0,2.027934,0.785398,0.785398,3.926991,3.926991,0.0001,0.0001,0.0001,0.0001,0.0,0.0,0.000000,0,0,0,0,0,0,0,0,0.00000,0.000000
4,0.08,8.443279,0.0,0.0,0.0,1.597808,0.785398,0.785398,3.926991,3.926991,0.0001,0.0001,0.0001,0.0001,0.0,0.0,0.000000,0,0,0,0,0,0,0,0,0.00000,0.000000


In [20]:
predDF.to_csv(os.path.join(dataOutput, "pathPrediction.csv"), index = False)

In [19]:
PredControl.loc[:, ["F_pred", "alpha_pred", "tau_pred"]]

,F_pred,alpha_pred,tau_pred
0,14022.81543,1.174873,-3468.092041


___

In [48]:
rowNum = 5

newRow = pd.DataFrame(np.array([
    -44.25776313553574,
 -815.4245488985321,
 55.146471684974486,
 639.3368156527224,
 2.049779294815162,
 20.336940630025254,
 5.23154914477651,
 20.674704195394455
]).reshape(1, -1), 
            columns = ["x_0", "x_dot_0", "y_0", "y_dot_0", "theta_0", "theta_dot_0", "phi_0", "phi_dot_0"])
newRow

,x_0,x_dot_0,y_0,y_dot_0,theta_0,theta_dot_0,phi_0,phi_dot_0
0,-44.257763,-815.424549,55.146472,639.336816,2.049779,20.336941,5.231549,20.674704


In [49]:

predDF.loc[[rowNum], ["x_0", "x_dot_0", "y_0", "y_dot_0", "theta_0", "theta_dot_0", "phi_0", "phi_dot_0"]] = np.array(newRow)
predDF.head()

,tsteps,signal,x_0,x_99,y_0,y_99,theta_0,theta_99,phi_0,phi_99,x_dot_0,y_dot_0,phi_dot_0,theta_dot_0,Fx,Fy,tau_pred,x_99_actual,xd_99_actual,y_99_actual,yd_99_actual,theta_99_actual,thetad_99_actual,phi_99_actual,phid_99_actual,F_pred,alpha_pred
0,0.00,0.000000,0.000000,0.0,0.000000,NaN,0.785398,0.785398,3.926991,3.926991,0.000100,0.000100,0.000100,0.000100,0.0,0.0,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000
1,0.02,2.486899,0.000000,0.0,0.000000,2.486899,0.785398,0.785398,3.926991,3.926991,0.000100,0.000100,0.000100,0.000100,0.0,0.0,-3468.092041,0,0,0,0,0,0,0,0,14022.815430,1.174873
2,0.04,4.817537,-1.004802,0.0,2.192336,2.330638,0.820536,0.785398,3.968901,3.926991,-104.001496,221.031005,4.163187,3.817106,0.0,0.0,-3585.508545,0,0,0,0,0,0,0,0,12486.833008,1.206250
3,0.06,6.845471,-17.936405,0.0,29.150296,2.027934,1.360996,0.785398,4.529490,3.926991,-497.219047,623.503133,14.484268,14.143846,0.0,0.0,-3753.160400,0,0,0,0,0,0,0,0,10318.933594,1.265749
4,0.08,8.443279,-29.498495,0.0,42.071666,1.597808,1.674637,0.785398,4.849789,3.926991,-675.001114,671.938846,17.897622,17.558187,0.0,0.0,-3960.925049,0,0,0,0,0,0,0,0,7705.878906,1.383812


In [50]:
# predict

nnpreds = model.predict(X_scaled[rowNum, :].reshape(-1,12))
nnpreds_unscaled = scalerY.inverse_transform(nnpreds)

# Y = testDF.loc[:, ["Fx", "Fy", "tau", "x_dot_99", "y_dot_99", 
#                    "phi_dot_99", "theta_dot_99"] ]


# these are input values
##         x,    xd,    y,   yd,    theta,  thetad,       phi,     phid,         F,          alpha,       tau0
print(np.array(X.loc[rowNum, ["x_0", "x_dot_0", "y_0", "y_dot_0", "theta_0", "theta_dot_0", "phi_0", "phi_dot_0"]]))

# this is for the next step -- they are predictions
print(pd.DataFrame(np.array(nnpreds_unscaled[0, [3,4,5,6]]).reshape(1, -1), columns =["x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"]))


[0.000000e+00 1.000000e-04 0.000000e+00 1.000000e-04 7.853980e-01
 1.000000e-04 3.926991e+00 1.000000e-04]
     x_dot_0    y_dot_0  phi_dot_0  theta_dot_0
0 -85.148544  64.816368  -0.096059    -0.636172


In [53]:
PredControl = pd.DataFrame(np.array(nnpreds_unscaled[0, [0,1,2]]).reshape(1, -1), columns = ["Fx", "Fy", "tau_pred"])
# convert to F, alpha, tau
PredControl["F_pred"] = np.sqrt(PredControl["Fx"]**2 + PredControl["Fy"]**2)

# calculate alpha
def quadrant(Fx, Fy):
    if (Fx >= 0) & (Fy >= 0):
        q = 1
    elif (Fx < 0) & (Fy >= 0):
        q = 2
    elif (Fx < 0) & (Fy < 0):
        q = 3
    elif (Fx >= 0) & (Fy < 0):
        q = 4
    else:
        q = 999999
    return(q)

PredControl["quadrant"] = [quadrant(fx, fy) for fx,fy in zip(PredControl["Fx"], PredControl["Fy"])]

def angleCalc(Fx, Fy, q):
    fx = np.abs(Fx)
    fy = np.abs(Fy)
    
    if q == 1:
        alpha = np.arctan(fy/fx)
    elif q == 2:
        alpha = np.pi - np.arctan(fy/fx)
    elif q == 3: 
        alpha = np.pi + np.arctan(fy/fx)
    elif q == 4:
        alpha = (2*np.pi) - np.arctan(fy/fx)
    return(alpha)

PredControl["alpha_pred"] = [angleCalc(fx, fy, q) for fx, fy, q in zip(PredControl["Fx"], PredControl["Fy"], PredControl["quadrant"])]

predDF.loc[[rowNum], ["F_pred", "alpha_pred", "tau_pred"]] = np.array(PredControl.loc[:, ["F_pred", "alpha_pred", "tau_pred"]])
print(PredControl.loc[:, ["Fx", "Fy", "F_pred", "alpha_pred", "tau_pred"]])
predDF.head()

           Fx           Fy       F_pred  alpha_pred     tau_pred
0 -454.458527  4993.998047  5014.633301    1.661547 -4196.091797


,tsteps,signal,x_0,x_99,y_0,y_99,theta_0,theta_99,phi_0,phi_99,x_dot_0,y_dot_0,phi_dot_0,theta_dot_0,Fx,Fy,tau_pred,x_99_actual,xd_99_actual,y_99_actual,yd_99_actual,theta_99_actual,thetad_99_actual,phi_99_actual,phid_99_actual,F_pred,alpha_pred
0,0.00,0.000000,0.000000,0.0,0.000000,NaN,0.785398,0.785398,3.926991,3.926991,0.000100,0.000100,0.000100,0.000100,0.0,0.0,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000
1,0.02,2.486899,0.000000,0.0,0.000000,2.486899,0.785398,0.785398,3.926991,3.926991,0.000100,0.000100,0.000100,0.000100,0.0,0.0,-3468.092041,0,0,0,0,0,0,0,0,14022.815430,1.174873
2,0.04,4.817537,-1.004802,0.0,2.192336,2.330638,0.820536,0.785398,3.968901,3.926991,-104.001496,221.031005,4.163187,3.817106,0.0,0.0,-3585.508545,0,0,0,0,0,0,0,0,12486.833008,1.206250
3,0.06,6.845471,-17.936405,0.0,29.150296,2.027934,1.360996,0.785398,4.529490,3.926991,-497.219047,623.503133,14.484268,14.143846,0.0,0.0,-3753.160400,0,0,0,0,0,0,0,0,10318.933594,1.265749
4,0.08,8.443279,-29.498495,0.0,42.071666,1.597808,1.674637,0.785398,4.849789,3.926991,-675.001114,671.938846,17.897622,17.558187,0.0,0.0,-3960.925049,0,0,0,0,0,0,0,0,7705.878906,1.383812


In [54]:
predDF.to_csv(os.path.join(dataOutput, "pathPrediction.csv"), index = False)

___

In [ ]:
# put predictions back into data frame


predDF.head()

In [ ]:
predDF.loc[ii + 1, ["x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"]] = nnpreds_unscaled[0, [3,4,5,6]]
predDF.loc[ii, ["Fx", "Fy", "tau_pred"]] = nnpreds_unscaled[0, [0,1,2]]

X = predDF.loc[1:, ["x_0", "y_0", "phi_0", "theta_0", 
            "x_99", "y_99", "phi_99", "theta_99", 

                    "x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"]]
X_scaled = scalerX.transform(X)

In [ ]:
# Predict steps with nnet, and reset at actual value (from simulation)
for ii in range(1, len(tsteps)-1):
    nnpreds = model.predict(X_scaled[ii, :].reshape(-1,12))
    nnpreds_unscaled = scalerY.inverse_transform(nnpreds)

    # Y = testDF.loc[:, ["Fx", "Fy", "tau", "x_dot_99", "y_dot_99", 
    #                    "phi_dot_99", "theta_dot_99"] ]
    
   
    # put predictions back into data frame
    predDF.loc[ii + 1, ["x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"]] = nnpreds_unscaled[0, [3,4,5,6]]
    predDF.loc[ii, ["Fx", "Fy", "tau_pred"]] = nnpreds_unscaled[0, [0,1,2]]
    
    X = predDF.loc[1:, ["x_0", "y_0", "phi_0", "theta_0", 
                "x_99", "y_99", "phi_99", "theta_99", 
               "x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"]]
    X_scaled = scalerX.transform(X)
    

In [ ]:
Xdf = pd.DataFrame(X_scaled, columns = ["x_0", "y_0", "phi_0", "theta_0", 
                "x_99", "y_99", "phi_99", "theta_99", 
               "x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"])

In [ ]:
predDF.head()

In [ ]:
# convert to F, alpha, tau
predDF["F_pred"] = np.sqrt(predDF["Fx"]**2 + predDF["Fy"]**2)

# calculate alpha
def quadrant(Fx, Fy):
    if (Fx >= 0) & (Fy >= 0):
        q = 1
    elif (Fx < 0) & (Fy >= 0):
        q = 2
    elif (Fx < 0) & (Fy < 0):
        q = 3
    elif (Fx >= 0) & (Fy < 0):
        q = 4
    else:
        q = 999999
    return(q)

predDF["quadrant"] = [quadrant(fx, fy) for fx,fy in zip(predDF["Fx"], predDF["Fy"])]

def angleCalc(Fx, Fy, q):
    fx = np.abs(Fx)
    fy = np.abs(Fy)
    
    if q == 1:
        alpha = np.arctan(fy/fx)
    elif q == 2:
        alpha = np.pi - np.arctan(fy/fx)
    elif q == 3: 
        alpha = np.pi + np.arctan(fy/fx)
    elif q == 4:
        alpha = (2*np.pi) - np.arctan(fy/fx)
    return(alpha)

predDF["alpha_pred"] = [angleCalc(fx, fy, q) for fx, fy, q in zip(predDF["Fx"], predDF["Fy"], predDF["quadrant"])]

predDF.head()

In [ ]:
# save predicted DF
predDF.to_csv(os.path.join(dataOutput, "PredictedTraj2.csv"), index = False)


In [ ]:
predDF.iloc[1, :]

In [ ]:
trainDF2 = pd.read_csv(os.path.join(dataDir2, "smallDF.csv"))
trainDF2.head()

In [ ]:
# make dataset
indexCut = int(0.9 * trainDF2.shape[0])
trainDF2["Test_Set"] = trainDF2.index > indexCut

# refref: add dots as output

X = trainDF2.loc[0:indexCut-1, ["x_0", "y_0", "phi_0", "theta_0", 
                    "x_99", "y_99", "phi_99", "theta_99", 
                   "x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"]]

Y = trainDF2.loc[0:indexCut-1, ["Fx", "Fy", "tau", "x_dot_99", "y_dot_99", 
                    "phi_dot_99", "theta_dot_99"] ]

# # # subtract x_0 and y_0 from everything
# X.x_99 = X.x_99 - X.x_0
# X.y_99 = X.y_99 - X.y_0

# X.x_0 = X.y_0 = 0

In [ ]:
# plot range of initial conditions (scatterplot matrix)
sns.pairplot(trainDF2.loc[:, ["x_0", "y_0", "phi_0", "theta_0", "Test_Set"]].drop_duplicates(), 
             vars = ["x_0", "y_0", "phi_0", "theta_0"], 
             hue = "Test_Set", diag_kind = 'kde', 
            palette = 'viridis', plot_kws=dict(alpha = 0.5, s = 20, edgecolor="grey", linewidth=0))
plt.savefig(os.path.join(figDir, "TrainingVSTestConditions.png"), width = 10, height = 10, dpi = 120)

In [ ]:
X.shape

In [ ]:
XTEST = trainDF2.loc[indexCut:, ["x_0", "y_0", "phi_0", "theta_0", 
                    "x_99", "y_99", "phi_99", "theta_99", 
                   "x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"]]

YTEST = trainDF2.loc[indexCut:, ["Fx", "Fy", "tau", "x_dot_99", "y_dot_99", 
                    "phi_dot_99", "theta_dot_99"] ]

In [ ]:
# see how much test data is in the training data
mergedDF = pd.merge(XTEST.iloc[:, 0:4].drop_duplicates(), X.iloc[:,0:4].drop_duplicates(), how= "inner")

print(mergedDF.shape)

mergedDF.head()

In [ ]:
XTEST.shape

In [ ]:
X.head()

In [ ]:
Y.head()

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.9, random_state = 123)

In [ ]:
from sklearn.preprocessing import StandardScaler  
scalerX = StandardScaler()  
scalerY = StandardScaler()  

# Don't cheat - fit only on training data
scalerX.fit(Xtrain)  
scalerY.fit(Ytrain) 

Xtrain_scaled = scalerX.transform(Xtrain)  
Ytrain_scaled = scalerY.transform(Ytrain)  

# apply same transformation to test data
Xtest_scaled = scalerX.transform(Xtest)
Ytest_scaled = scalerY.transform(Ytest)

# apply to final test set (novel initial conditions)
XTEST_scaled = scalerX.transform(XTEST)
YTEST_scaled = scalerY.transform(YTEST)

In [ ]:
print(Xtest.shape)
pd.DataFrame(scalerY.inverse_transform(Ytrain_scaled)).head()

In [ ]:
pd.DataFrame(Ytrain).reset_index(drop = True).head()

In [ ]:
# from sklearn.preprocessing import MinMaxScaler  
# scalerX = MinMaxScaler()  
# scalerY = MinMaxScaler()  

# # Don't cheat - fit only on training data
# scalerX.fit(Xtrain)  
# scalerY.fit(Ytrain) 

# Xtrain_scaled = scalerX.transform(Xtrain)  
# Ytrain_scaled = scalerY.transform(Ytrain)  

# # apply same transformation to test data
# Xtest_scaled = scalerX.transform(Xtest)
# Ytest_scaled = scalerY.transform(Ytest)

In [ ]:
# build Keras model with functional API, instead of Sequential
from keras.layers import Input, Dense
from keras.models import Model

# callcacks
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau

In [ ]:
inputs = Input(shape=(Xtrain_scaled.shape[1],))

x = Dense(400, activation='tanh')(inputs)
x = Dense(400, activation='tanh')(x)
x = Dense(400, activation='tanh')(x)
x = Dense(16, activation='tanh')(x)
predictions = Dense(Ytrain_scaled.shape[1], activation='linear')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(loss="mse", optimizer="adam", metrics = ['mse'])

In [ ]:
earlystop = EarlyStopping(monitor='val_mean_squared_error', patience=50, 
                          verbose=1, mode='auto', min_delta = 0.0001)

history = model.fit(Xtrain_scaled, Ytrain_scaled, validation_split = 0.7, 
                    verbose = 2, batch_size=2**14, epochs = 1000, 
                    callbacks = [earlystop])

In [ ]:
def plot_model_history(model_history, saveFig = False):
    fig, axs = plt.subplots(1,1,figsize=(10,5))
    # summarize history for accuracy
    axs.plot(range(1,len(model_history.history['mean_squared_error'])+1),
             model_history.history['mean_squared_error'])
    axs.plot(range(1,len(model_history.history['val_mean_squared_error'])+1),
             model_history.history['val_mean_squared_error'])
    axs.set_title('Model MSE')
    axs.set_ylabel('mean_squared_error')
    axs.set_xlabel('Epoch')
    axs.set_xticks(np.arange(1,len(model_history.history['val_mean_squared_error'])+1),
                   len(model_history.history['val_mean_squared_error'])/10)
    axs.legend(['train', 'val'], loc='best')
    if saveFig:
        fig.savefig(os.path.join(figDir, "ModelTraining_HoldoutData.png"), dpi = 120, bbox_inches='tight')
    plt.show()

In [ ]:
plot_model_history(history, saveFig = False)

In [ ]:
# save model
# model.save(os.path.join(savedModels, 'my_model_dataHoldout.h5'))
# aa = r'explorer /select,"'
# subprocess.Popen(str(aa + savedModels + '\"'))

In [ ]:
# # load model
from keras.models import load_model
model = load_model(os.path.join(savedModels, 'my_model_dataHoldout.h5'))

In [ ]:
nnpreds = model.predict(Xtest_scaled)

In [ ]:
# rescale
nnpreds_unscaled = scalerY.inverse_transform(nnpreds)

# Plot on randomly-withheld data (initial conditions in the test data)

In [ ]:
plt.close("all")
fig, axs = plt.subplots(2,4, figsize=(15, 10), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.3, wspace=0.3)
fig.suptitle('Predicted vs. acutal for randomly withheld data', fontsize=14, fontweight='bold')

axs = axs.ravel()

nms = Y.columns

# replace lightest colors with white
import matplotlib.colors

cmap = plt.cm.magma_r
cmaplist = np.array([cmap(i) for i in range(cmap.N)])
#cmaplist[0:2] = np.repeat((0.0,0.0,0.0,0.0), 2).reshape(-1,4)
cmaplist[:,0:3] = np.divide(cmaplist[:, 0:3], 1.1)
#cmaplist[:,0:3] = np.transpose(np.vstack([np.linspace(1, 0, 256), np.linspace(1, 0, 256), np.linspace(1, 0, 256)]))
cmaplist[0] = (1,1,1,0.5)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, cmap.N)

for ii in range(len(Y.columns)+1):
    # read in image
    
    try:
        axs[ii].hexbin(y = Ytest.iloc[:,ii],x = nnpreds_unscaled[:,ii], gridsize = 150, cmap = cmap)
        axs[ii].set_xlabel("Predicted Value\n(unscaled)")
        if(ii == 0):
            axs[ii].set_ylabel("Actual Value\n(unscaled)")
        axs[ii].set_title(nms[ii])
        axs[ii].plot(Ytest.iloc[:,ii], Ytest.iloc[:,ii], 'grey', linewidth = 1, linestyle  = "--")
    except:
        pass
    if ii == 7:
        axs[ii].set_visible(False)

fig.savefig(os.path.join(figDir, "NNPreds_newData_control_bigbatch_FxFy_hexbin_holdoutData.png"), dpi = 120, bbox_inches='tight')

In [ ]:
# residual plots
plt.close("all")
fig, axs = plt.subplots(2,4, figsize=(15, 10), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.3, wspace=0.3)
fig.suptitle('Residual Plots for randomly withheld data ', fontsize=14, fontweight='bold')

axs = axs.ravel()

nms = Y.columns

# replace lightest colors with white
import matplotlib.colors

cmap = plt.cm.magma_r
cmaplist = np.array([cmap(i) for i in range(cmap.N)])
cmaplist[:,0:3] = np.divide(cmaplist[:, 0:3], 1.1)
cmaplist[0] = (1,1,1,0.5)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, cmap.N)

for ii in range(len(Y.columns) + 1):
    # read in image
    
    try:
        axs[ii].hexbin(y = Ytest.iloc[:,ii] - nnpreds_unscaled[:,ii],x = nnpreds_unscaled[:,ii], gridsize = 150, cmap = cmap)
        axs[ii].set_xlabel("Predicted Value\n(unscaled)")
        if(ii == 0) or (ii == 4):
            axs[ii].set_ylabel("Actual - Predicted Value\n(unscaled)")
        axs[ii].set_title(nms[ii])
        axs[ii].hlines(y = 0, xmin = np.min(nnpreds_unscaled[:,ii]), 
                       xmax = np.max(nnpreds_unscaled[:,ii]), linestyle =  "--", linewidth = 1)
    except:
        pass
    if ii == 7:
        axs[ii].set_visible(False)
        
fig.savefig(os.path.join(figDir, "residual_Hexbin_holdoutData.png"), dpi = 120, bbox_inches='tight')

# Plot for novel test data (initial conditions not found in training, but coming from same distribution)




In [ ]:
# predict on holdout data (novel initial conditions)
nnpreds2 = model.predict(XTEST_scaled)

In [ ]:
nnpreds_unscaled = scalerY.inverse_transform(nnpreds2)

In [ ]:
plt.close("all")
fig, axs = plt.subplots(2,4, figsize=(15, 10), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.3, wspace=0.3)
fig.suptitle('Predicted vs. acutal for novel initial conditions \n from same distribution as original data', fontsize=14, fontweight='bold')

axs = axs.ravel()

nms = Y.columns

# replace lightest colors with white
import matplotlib.colors

cmap = plt.cm.magma_r
cmaplist = np.array([cmap(i) for i in range(cmap.N)])
#cmaplist[0:2] = np.repeat((0.0,0.0,0.0,0.0), 2).reshape(-1,4)
cmaplist[:,0:3] = np.divide(cmaplist[:, 0:3], 1.1)
#cmaplist[:,0:3] = np.transpose(np.vstack([np.linspace(1, 0, 256), np.linspace(1, 0, 256), np.linspace(1, 0, 256)]))
cmaplist[0] = (1,1,1,0.5)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, cmap.N)

for ii in range(len(Y.columns)+1):
    # read in image
    
    try:
        axs[ii].hexbin(y = YTEST.iloc[:,ii],x = nnpreds_unscaled[:,ii], gridsize = 150, cmap = cmap)
        axs[ii].set_xlabel("Predicted Value\n(unscaled)")
        if(ii == 0) or (ii == 4):
            axs[ii].set_ylabel("Actual Value\n(unscaled)")
        axs[ii].set_title(nms[ii])
        axs[ii].plot(YTEST.iloc[:,ii], YTEST.iloc[:,ii], 'grey', linewidth = 1, linestyle  = "--")
    except:
        pass
    if ii == 7:
        axs[ii].set_visible(False)

fig.savefig(os.path.join(figDir, "NNPreds_novelData.png"), dpi = 120, bbox_inches='tight')

In [ ]:
# residual plots
plt.close("all")
fig, axs = plt.subplots(2,4, figsize=(15, 10), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.3, wspace=0.3)
fig.suptitle('Residual Plots for novel initial conditions \n from same distribution as original data', fontsize=14, fontweight='bold')

axs = axs.ravel()

nms = Y.columns

# replace lightest colors with white
import matplotlib.colors

cmap = plt.cm.magma_r
cmaplist = np.array([cmap(i) for i in range(cmap.N)])
cmaplist[:,0:3] = np.divide(cmaplist[:, 0:3], 1.1)
cmaplist[0] = (1,1,1,0.5)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, cmap.N)

for ii in range(len(Y.columns) + 1):
    # read in image
    
    try:
        axs[ii].hexbin(y = YTEST.iloc[:,ii] - nnpreds_unscaled[:,ii],x = nnpreds_unscaled[:,ii], gridsize = 150, cmap = cmap)
        axs[ii].set_xlabel("Predicted Value\n(unscaled)")
        if(ii == 0) or (ii == 4):
            axs[ii].set_ylabel("Actual - Predicted\nValue (unscaled)")
        axs[ii].set_title(nms[ii])
        axs[ii].hlines(y = 0, xmin = np.min(nnpreds_unscaled[:,ii]), 
                       xmax = np.max(nnpreds_unscaled[:,ii]), linestyle =  "--", linewidth = 1)
    except:
        pass
    if ii == 7:
        axs[ii].set_visible(False)
        
fig.savefig(os.path.join(figDir, "residual_Hexbin_NovelData.png"), dpi = 120, bbox_inches='tight')

In [ ]:
# residual plots
plt.close("all")
fig, axs = plt.subplots(2,4, figsize=(15, 10), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.3, wspace=0.3)
fig.suptitle('Residual Plots for novel initial conditions \n from same distribution as original data', fontsize=14, fontweight='bold')

axs = axs.ravel()

nms = Y.columns

# replace lightest colors with white
import matplotlib.colors

cmap = plt.cm.magma_r
cmaplist = np.array([cmap(i) for i in range(cmap.N)])
cmaplist[:,0:3] = np.divide(cmaplist[:, 0:3], 1.1)
cmaplist[0] = (1,1,1,0.5)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, cmap.N)

for ii in range(len(Y.columns) + 1):
    # read in image
    
    try:
        axs[ii].scatter(y = YTEST.iloc[:,ii] - nnpreds_unscaled[:,ii],x = nnpreds_unscaled[:,ii], s = 1, alpha = 0.1 )
        axs[ii].set_xlabel("Predicted Value\n(unscaled)")
        if(ii == 0):
            axs[ii].set_ylabel("Actual Value\n(unscaled)")
        axs[ii].set_title(nms[ii])
        axs[ii].hlines(y = 0, xmin = np.min(nnpreds_unscaled[:,ii]), 
                       xmax = np.max(nnpreds_unscaled[:,ii]), linestyle =  "--", linewidth = 1)
    except:
        pass
    if ii == 7:
        axs[ii].set_visible(False)
        
fig.savefig(os.path.join(figDir, "residual_scatter_NovelData.png"), dpi = 120, bbox_inches='tight')

# predict path for last 4 datafiles and compare to original

In [ ]:
dataFiles[-4:]

In [ ]:
# dfList = []

# for dataFile in dataFiles:
#     bf2 = pd.read_hdf(dataFile, "data")
#     bf2[['simNum', 'timestep']] = bf2[['simNum', 'timestep']].astype(int)
#     bf2['count'] = bf2.groupby(["simNum", "timestep"]).cumcount()
#     sdf = bf2.loc[(bf2["count"] == 0) | (bf2["count"] == 99), :]
#     ss = sdf.pivot_table(index=['simNum', 'timestep'], columns = "count",aggfunc='first')
#     ss.columns = [f'{i}_{j}' if j != '' else f'{i}' for i,j in ss.columns]
#     ss.reset_index(inplace=True) 
#     dfList.append(ss)
#     print(dataFile)

In [ ]:
bf2 = pd.read_hdf(dataFiles[-1], "data")

In [ ]:
bf2.head()

In [ ]:
dataDir

In [ ]:
Qmat = scipy.io.loadmat(os.path.join(dataDir, "Qstore_6_hws_am_con"+ ".mat"))

In [ ]:
for timestep in range(100):
    if np.mod(timestep, 10) == 0: 
        print(timestep)
    vals = Qmat['Qstore'][timestep,0] #<-- set the array you want to access. 
    keys = Qmat['Qstore'][timestep,0].dtype.descr
    keys = np.array(keys)[:, 0]

# Assemble the keys and values into variables with the same name as that used in MATLAB
for i in range(len(keys)):
    key = keys[i]
    val = np.squeeze(vals[key][0])  # squeeze is used to covert matlat (1,n) arrays into numpy (1,) arrays. 
    exec(key + '=val')


In [ ]:
keys

In [ ]:
stt = time.time()
tt = np.zeros((100, 15))
bigF = np.zeros((100*2500*100, 15))
ctr = 0
for timestep in range(20):
    if np.mod(timestep, 10) == 0: 
        print(timestep)
    vals = Qmat['Qstore'][timestep,0] #<-- set the array you want to access. 
    keys = Qmat['Qstore'][timestep,0].dtype.descr
    keys = np.array(keys)[:, 0]

    # Assemble the keys and values into variables with the same name as that used in MATLAB
    for i in range(len(keys)):
        key = keys[i]
        val = np.squeeze(vals[key][0])  # squeeze is used to covert matlat (1,n) arrays into numpy (1,) arrays. 
        exec(key + '=val')

    for simNum in range(2500):
        #print(timestep, simNum)
        ttRows = tt.shape[0]
        tt[:, 0:8] = vals["bigQ"][0, simNum]
        tt[:, [8]] = np.repeat(np.concatenate(vals["F"][0]).ravel()[simNum], ttRows).reshape(-1,1)
        tt[:, [9]] = np.repeat(np.concatenate(vals["alpha"][0]).ravel()[simNum], ttRows).reshape(-1,1)
        tt[:, [10]] = np.repeat(np.concatenate(vals["tau0"][0]).ravel()[simNum], ttRows).reshape(-1,1)
        tt[:, [11]] =  np.repeat(simNum, ttRows).reshape(-1,1)
        tt[:, [12]] = np.repeat(timestep, ttRows).reshape(-1,1)
        tt[:, [13]] = np.repeat(np.concatenate(vals["cost"][0]).ravel()[simNum], ttRows).reshape(-1,1)
        tt[:, [14]] =  np.arange(0, ttRows).reshape(-1,1)
        bigF[ctr:ctr+ttRows, :] = tt
        ctr += ttRows

print(time.time() - stt )


In [ ]:
tt2 = pd.DataFrame(bigF, columns=["x", "y", "theta", "phi", "x_dot", "y_dot", "theta_dot", "phi_dot", 
                                       "F", "alpha", "tau", "simNum", "timestep", "cost", "microStepNum"])

In [ ]:
tt2.head()

In [ ]:
# find minimum cost for each timestep

#remove 0's
tt2 = tt2.loc[~(tt2==0).all(axis=1)]

tt2.shape

In [ ]:
tmp = tt2.loc[tt2.groupby(["timestep"])["cost"].idxmin(), :]
tmp

In [ ]:
t3 = tt2.loc[tt2.cost.isin(tmp.cost),:].copy()

In [ ]:
t3.head()

In [ ]:
t3.shape

In [ ]:
tt2.shape

In [ ]:
t4 = tt2.sort_values("cost")

In [ ]:
# t4.groupby("cost").count()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(t3.x, t3.y)
ax.set_aspect('equal', 'datalim')

In [ ]:
t3.reset_index(inplace = True)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(y = t3.y, x = t3.reset_index().index)
#ax.set_aspect('equal', 'datalim')

In [ ]:
# plot nnet predictions over the top
t3.loc[:,['simNum', 'timestep']] = t3.loc[:,['simNum', 'timestep']].astype(int).copy()

In [ ]:
t3['count'] = t3.groupby(["simNum", "timestep"]).cumcount().copy()
sdf = t3.loc[(t3["count"] == 0) | (t3["count"] == 99), :].copy()
ss = sdf.pivot_table(index=['simNum', 'timestep'], columns = "count",aggfunc='first').copy()
ss.columns = [f'{i}_{j}' if j != '' else f'{i}' for i,j in ss.columns]
ss.reset_index(inplace=True) 
ss.head()

In [ ]:
ss["y_0"]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ss.sort_values("timestep", inplace=True)
ss.reset_index(inplace = True, drop = True)
ax.scatter(y = ss.y_0, x = ss.timestep)
ax.scatter(y = ss.y_99, x = ss.timestep + 1)

In [ ]:
# predict

# clean data
ss = ss.drop(columns=['F_99', 'alpha_99', 
                               'tau_99'])
ss=ss.rename(columns = {'F_0':'F', 
                                 "alpha_0": "alpha", 
                                 "tau_0": "tau"})

In [ ]:
# convert to Fx and Fy
ss["Fx"] = ss.F * np.cos(ss.alpha)
ss["Fy"] = ss.F * np.sin(ss.alpha)

# make dataset
XTEST_traj = ss.loc[:, ["x_0", "y_0", "phi_0", "theta_0", 
                    "x_99", "y_99", "phi_99", "theta_99", 
                   "x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"]]

YTEST_traj = ss.loc[:, ["Fx", "Fy", "tau", "x_dot_99", "y_dot_99", 
                    "phi_dot_99", "theta_dot_99"] ]


# scale
XTEST_traj_sc = scalerX.transform(XTEST_traj)
YTEST_traj_sc = scalerY.transform(YTEST_traj)


In [ ]:
# predict
sc_preds = model.predict(XTEST_traj_sc)

In [ ]:
# rescale
preds_unsc = scalerY.inverse_transform(sc_preds)

In [ ]:
# show predictions
pd.DataFrame(preds_unsc, columns = YTEST_traj.columns).head()

In [ ]:
# show actual
YTEST_traj.head()

In [ ]:
XTEST_traj.head()

In [ ]:
freq = 1/25
timesteps = np.arange(0, 20)

goal = 10*np.sin(2*np.pi*freq * timesteps)

# goal deriv
goalD = 4/5*np.pi *np.cos(2*np.pi*freq * timesteps)


plt.scatter(y = goal, x = timesteps)
plt.scatter(y = goalD, x = timesteps)
plt.hlines(y = 0, xmin = 0, xmax = 20)
#plt.scatter(y = ss.y_0, x = ss.timestep)

In [ ]:
# goal deriv
goalD = 4/5*np.pi *np.cos(2*np.pi*freq * timesteps)

In [ ]:
XTEST_traj.head()

In [ ]:
# create input data
theta_goal = np.pi / 4
phi_goal = np.pi / 4 + np.pi

timstep = 0
inputData = np.array([0, goal[timstep], phi_goal, theta_goal, 
             0, goal[timstep + 1], phi_goal, theta_goal,
            0.0001, 0.0001, 0, 0 ]).reshape(1,-1)

#inputData2 = scalerX.transform(XTEST_traj.head())
inputData2 = scalerX.transform(inputData)
pd.DataFrame(inputData2, columns = XTEST_traj.columns)

In [ ]:
nnpreds2 = pd.DataFrame(scalerY.inverse_transform(model.predict(inputData2)), columns = YTEST_traj.columns)

In [ ]:
newDF = pd.concat([pd.DataFrame(inputData, columns = XTEST_traj.columns), nnpreds2], axis = 1)

In [ ]:
for timstep in np.arange(0,16): 
    newDF = newDF.append(newDF.iloc[timstep,:]).reset_index(drop = True)
    newDF.loc[timstep+1, ['x_0', 'y_0', 'phi_0', 'theta_0']] = np.array(newDF.loc[timstep, ['x_99', 'y_99', 'phi_99', 'theta_99']])
    newDF.loc[timstep+1, ['x_99', 'y_99', 'phi_99', 'theta_99']] = np.array([0, goal[timstep + 2], phi_goal, theta_goal])
    newDF.loc[timstep+1, ['x_dot_0', 'y_dot_0', 'phi_dot_0', 'theta_dot_0']] = np.array(newDF.loc[timstep, ['x_dot_99', 'y_dot_99', 'phi_dot_99', 'theta_dot_99']])
    inputData = np.array(newDF.loc[timstep + 1,list(XTEST_traj.columns)]).reshape(1,-1)
    inputData2 = scalerX.transform(inputData)

    newDF.loc[timstep + 1, list(YTEST_traj.columns)] = scalerY.inverse_transform(model.predict(inputData2))

In [ ]:
newDF

In [ ]:
# convert to F and alpha 

# calculate alpha
def quadrant(Fx, Fy):
    if (Fx >= 0) & (Fy >= 0):
        q = 1
    elif (Fx < 0) & (Fy >= 0):
        q = 2
    elif (Fx < 0) & (Fy < 0):
        q = 3
    elif (Fx >= 0) & (Fy < 0):
        q = 4
    else:
        q = 999999
    return(q)


def angleCalc(Fx, Fy, q):
    fx = np.abs(Fx)
    fy = np.abs(Fy)
    
    if q == 1:
        alpha = np.arctan(fy/fx)
    elif q == 2:
        alpha = np.pi - np.arctan(fy/fx)
    elif q == 3: 
        alpha = np.pi + np.arctan(fy/fx)
    elif q == 4:
        alpha = (2*np.pi) - np.arctan(fy/fx)
    return(alpha)

def AlphaAndForceCalc (df = newDF, FX_name = "Fx", FY_name = "Fy"):
    quadrantNum = [quadrant(fx, fy) for fx,fy in zip(df[FX_name], df[FY_name])]
    df["alphaCalc"] = [angleCalc(fx, fy, q) for fx, fy, q in zip(df[FX_name], df[FY_name], quadrantNum)]
    df["F"] = np.sqrt(df[FX_name]**2 + df[FY_name]**2)
    return(df)

# calculate F

df2 = AlphaAndForceCalc()

In [ ]:
df2

In [ ]:
df2.to_csv(os.path.join(dataOutput, "predictPartPath_allCols.csv"), index = False)

In [ ]:
# make smaller version
df2.drop(["Fx", "Fy"], axis = 1, inplace = True)
df2.rename({"alphaCalc": "alpha"}, axis = 1, inplace = True)
df2

In [ ]:
df2.columns

In [ ]:
# change column order

ddf1 = df2.pop("alpha")
df2['alpha'] = ddf1
df1 = df2.pop('tau')
df2['tau'] = df1
df2

In [ ]:
df2.to_csv(os.path.join(dataOutput, "predictPartPath_small.csv"), index = False)

In [ ]:
dataOutput

In [ ]:
YTEST_traj.head()

In [ ]:
newDF = newDF.loc[0:YTEST_traj.shape[0], YTEST_traj.columns]

newDF.head()

In [ ]:
plt.scatter(YTEST_traj.Fx, newDF.Fx)

In [ ]:
plt.scatter(YTEST_traj.Fy, newDF.Fy)

In [ ]:
plt.plot(YTEST_traj.Fx- newDF.Fx)

In [ ]:
plt.plot(YTEST_traj.Fy- newDF.Fy)

In [ ]:
# convert Fx and Fy back to F and alpha

In [ ]:
# calculate mean squared error

In [ ]:
from sklearn.metrics import mean_squared_error

from sklearn.metrics import r2_score

In [ ]:
for ii in range(len(nms)):
    #np.sqrt(mean_squared_error(Ytest.iloc[0:1000,ii], nnpreds_unscaled[:,ii]))
    print(r2_score(Ytest.iloc[0:1000,ii], nnpreds_unscaled[:,ii]), nms[ii])

In [ ]:
plt.scatter(nnpreds_unscaled[:,ii],Ytest.iloc[0:1000,ii]-  nnpreds_unscaled[:,ii])

In [ ]:
# predict on holdout data (novel initial conditions)
nnpreds = model.predict(XTEST_scaled)

nnpreds_unscaled = scalerY.inverse_transform(nnpreds)

In [ ]:
XTEST_scaled.shape

In [ ]:
YTEST.shape

In [ ]:
plt.close("all")
fig, axs = plt.subplots(2,4, figsize=(15, 10), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.3, wspace=0.3)
fig.suptitle('Predicted vs. acutal ', fontsize=14, fontweight='bold')

axs = axs.ravel()

nms = Y.columns

# replace lightest colors with white
import matplotlib.colors

cmap = plt.cm.magma_r
cmaplist = np.array([cmap(i) for i in range(cmap.N)])
#cmaplist[0:2] = np.repeat((0.0,0.0,0.0,0.0), 2).reshape(-1,4)
cmaplist[:,0:3] = np.divide(cmaplist[:, 0:3], 1.1)
#cmaplist[:,0:3] = np.transpose(np.vstack([np.linspace(1, 0, 256), np.linspace(1, 0, 256), np.linspace(1, 0, 256)]))
cmaplist[0] = (1,1,1,0.5)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, cmap.N)

for ii in range(len(Y.columns)):
    # read in image
    
    try:
        axs[ii].hexbin(y = YTEST.iloc[:,ii],x = nnpreds_unscaled[:,ii], gridsize = 150, cmap = cmap)
        axs[ii].set_xlabel("Predicted Value\n(unscaled)")
        if(ii == 0):
            axs[ii].set_ylabel("Actual Value\n(unscaled)")
        axs[ii].set_title(nms[ii])
        axs[ii].plot(YTEST.iloc[:,ii], YTEST.iloc[:,ii], 'grey', linewidth = 1, linestyle  = "--")
    except:
        pass

# fig.savefig(os.path.join(figDir, "NNPreds_newData_control_bigbatch_FxFy_hexbin_loadedModel.png"), dpi = 120, bbox_inches='tight')

In [ ]:
# residual plots
plt.close("all")
fig, axs = plt.subplots(2,4, figsize=(15, 10), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.3, wspace=0.3)
fig.suptitle('Predicted vs. acutal ', fontsize=14, fontweight='bold')

axs = axs.ravel()

nms = Y.columns

# replace lightest colors with white
import matplotlib.colors

cmap = plt.cm.magma_r
cmaplist = np.array([cmap(i) for i in range(cmap.N)])
cmaplist[:,0:3] = np.divide(cmaplist[:, 0:3], 1.1)
cmaplist[0] = (1,1,1,0.5)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, cmap.N)

for ii in range(len(Y.columns)):
    # read in image
    
    try:
        axs[ii].hexbin(y = YTEST.iloc[:,ii] - nnpreds_unscaled[:,ii],x = nnpreds_unscaled[:,ii], gridsize = 150, cmap = cmap)
        axs[ii].set_xlabel("Predicted Value\n(unscaled)")
        if(ii == 0):
            axs[ii].set_ylabel("Residuals\n(unscaled)")
        axs[ii].set_title(nms[ii])
        axs[ii].hlines(y = 0, xmin = np.min(nnpreds_unscaled[:,ii]), 
                       xmax = np.max(nnpreds_unscaled[:,ii]), linestyle =  "--", linewidth = 1)
    except:
        pass


In [ ]:
# see how many initial conditions are in the TEST Set


In [ ]:
XTEST.iloc[:, 0:4].drop_duplicates().shape

In [ ]:
X.iloc[:,0:4].drop_duplicates().shape

In [ ]:
df1 = pd.DataFrame({"A": [4,5,5,6,6,5,5], 
                   "B": [4,3,3,5,5,3,3], 
                   "C": [3,4,4,6,7,4,4], 
                   "D": [6,8,2,6,4,2,2]})
df1["NM"] = "df1"


df2 = pd.DataFrame({"A": [4,5,5], 
                   "B": [4,3,3], 
                   "C": [4,4,4], 
                   "D": [7,2,2]})

df2["NM"] = "df2"

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.merge(df1, df2, on = ['A', 'B', 'C', 'D'], how = "inner")

In [ ]:
TestPreds = pd.DataFrame(nnpreds_unscaled, columns = Y.columns)
TestPreds.head()

In [ ]:
TestInput = pd.DataFrame(Xtest, columns = X.columns).reset_index(drop = True)
TestInput.head()

In [ ]:
# F will be the same in any quadrant

TestPreds["F"] = np.sqrt(TestPreds["Fx"]**2 + TestPreds["Fy"]**2)

# trainDF["Fx"] = trainDF.F * np.cos(trainDF.alpha)
# trainDF["Fy"] = trainDF.F * np.sin(trainDF.alpha)

In [ ]:
TestPreds.head()

In [ ]:
# calculate alpha
def quadrant(Fx, Fy):
    if (Fx >= 0) & (Fy >= 0):
        q = 1
    elif (Fx < 0) & (Fy >= 0):
        q = 2
    elif (Fx < 0) & (Fy < 0):
        q = 3
    elif (Fx >= 0) & (Fy < 0):
        q = 4
    else:
        q = 999999
    return(q)


def angleCalc(Fx, Fy, q):
    fx = np.abs(Fx)
    fy = np.abs(Fy)
    
    if q == 1:
        alpha = np.arctan(fy/fx)
    elif q == 2:
        alpha = np.pi - np.arctan(fy/fx)
    elif q == 3: 
        alpha = np.pi + np.arctan(fy/fx)
    elif q == 4:
        alpha = (2*np.pi) - np.arctan(fy/fx)
    return(alpha)


In [ ]:
TestPreds["quadrant"] = [quadrant(fx, fy) for fx,fy in zip(TestPreds["Fx"], TestPreds["Fy"])]

In [ ]:
TestPreds.head()

In [ ]:
TestPreds["alpha"] = [angleCalc(fx, fy, q) for fx, fy, q in zip(TestPreds["Fx"], TestPreds["Fy"], TestPreds["quadrant"])]

In [ ]:
TestPreds.head()

In [ ]:
TestPredsComb = pd.concat([TestPreds, TestInput], axis = 1)

In [ ]:
TestPredsComb.head()

In [ ]:
TestPredsComb.rename(index=str, columns={"F": "F_pred", "alpha": "alpha_pred", "tau":"tau_pred"}, inplace=True)

In [ ]:
TestPredsComb.head()

In [ ]:
TestPredsComb.drop(['Fx', 'Fy', "quadrant"], axis=1, inplace = True)
TestPredsComb.head()

In [ ]:
cols= ['x_0', 'y_0', 'phi_0', 'theta_0', 
       'x_dot_0', 'y_dot_0', 'phi_dot_0', 'theta_dot_0', 
       'F_pred', 'alpha_pred', 'tau_pred', 
      'x_99', 'y_99', 'phi_99', 'theta_99', 
      'x_dot_99', 'y_dot_99', 'phi_dot_99', 'theta_dot_99' ]

In [ ]:
tc2 = TestPredsComb.loc[:, cols]
tc2.head()

In [ ]:
tc2.rename(columns = {"x_dot_99": "x_dot_99_pred", 
                     "y_dot_99": "y_dot_99_pred", 
                     "theta_dot_99": "theta_dot_99_pred", 
                     "phi_dot_99": "phi_dot_99_pred"}, inplace = True)
tc2.head()

In [ ]:
tc2.to_csv(os.path.join(dataOutput, "NNpreds.csv"), index = False)

In [ ]:
tc2.shape

In [ ]:
Xtrain.shape

In [ ]:

tc_sm =  tc2.sample(n=1000, random_state = 902902)
tc_sm.to_csv(os.path.join(dataOutput, "NNpreds_small.csv"), index = False)

In [ ]:
# check alpha on trainDf
trainDF2["quadrant"] = [quadrant(fx, fy) for fx,fy in zip(trainDF2["Fx"], trainDF2["Fy"])]

trainDF2["alphaCalc"] = [angleCalc(fx, fy, q) for fx, fy, q in zip(trainDF2["Fx"], trainDF2["Fy"], trainDF2["quadrant"])]

In [ ]:
trainDF2.loc[:, ["alpha", "alphaCalc"]].head()

In [ ]:
plt.close("all")
fig, axs = plt.subplots(2,int(len(Y.columns)/2), figsize=(15, 10), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.1, wspace=0.3)
fig.suptitle('Predicted vs. acutal ', fontsize=14, fontweight='bold')

axs = axs.ravel()

nms = Y.columns

for ii in range(len(Y.columns)):
    # read in image
    
    try:
        axs[ii].scatter(y = Ytest.iloc[:,ii],x = nnpreds_unscaled[:,ii], s = 1, alpha = 0.1 )
        axs[ii].set_xlabel("Predicted Value (un-scaled)")
        if(ii == 0):
            axs[ii].set_ylabel("Actual Value\n(un-scaled)")
        axs[ii].set_title(nms[ii])
        axs[ii].plot(Ytest.iloc[:,ii], Ytest.iloc[:,ii], 'b')
    except:
        pass

    
fig.savefig(os.path.join(figDir, "NNPreds_newData_control_bigbatch_FxFy.png"), dpi = 120, bbox_inches='tight')

In [ ]:
Ytest.head()

In [ ]:
Xtest.head()

In [ ]:
# refref: idea - predict whole paths, rather than just the endpoint

In [ ]:
plt.close("all")
fig, axs = plt.subplots(1,3, figsize=(15, 5), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.1, wspace=0.3)
fig.suptitle('Predicted vs. acutal ', fontsize=14, fontweight='bold')

axs = axs.ravel()

nms = ["Fx", "Fy", "tau"]


# replace lightest colors with white
import matplotlib.colors

cmap = plt.cm.magma_r
cmaplist = np.array([cmap(i) for i in range(cmap.N)])
#cmaplist[0:2] = np.repeat((0.0,0.0,0.0,0.0), 2).reshape(-1,4)
cmaplist[:,0:3] = np.divide(cmaplist[:, 0:3], 1.1)
#cmaplist[:,0:3] = np.transpose(np.vstack([np.linspace(1, 0, 256), np.linspace(1, 0, 256), np.linspace(1, 0, 256)]))
cmaplist[0] = (1,1,1,0.5)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, cmap.N)

for ii in range(3):
    # read in image
    
    try:
        axs[ii].hexbin(y = Ytest_scaled[:,ii],x = nnpreds[:,ii], gridsize = 150, cmap = cmap)
        axs[ii].set_xlabel("Predicted Value (scaled)")
        if(ii == 0):
            axs[ii].set_ylabel("Actual Value\n(scaled)")
        axs[ii].set_title(nms[ii])
        axs[ii].plot(Ytest_scaled[:,ii], Ytest_scaled[:,ii], 'grey', linewidth = 1, linestyle  = "--")
    except:
        pass

fig.savefig(os.path.join(figDir, "NNPreds_newData_control_bigbatch_FxFy_hexbin.png"), dpi = 120, bbox_inches='tight')

In [ ]:
bf2[['simNum', 'timestep']] = bf2[['simNum', 'timestep']].astype(int)

bf2['count'] = bf2.groupby(["simNum", "timestep"]).cumcount()

bf2.head()

bf2.tail()

sdf = bf2.loc[(bf2["count"] == 0) | (bf2["count"] == 99), :]
sdf.head()

ss = sdf.pivot_table(index='simNum', columns = "count",aggfunc='first')
ss.columns = [f'{i}_{j}' if j != '' else f'{i}' for i,j in ss.columns]
ss["simNum"] = ss.index
ss.index.name = None
ss.head()

np.array(ss).shape

np.vstack([np.array(ss), np.array(ss)]).shape